<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/DiffraxPID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install diffrax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.8/159.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 16.8 MB/s eta 0:00:00


In [54]:
from diffrax import diffeqsolve, ODETerm, Dopri5
import jax.numpy as jnp
import jax

In [152]:
a=0.01
A = 1.
qin_initial_ss = 0.1
sq2g = jnp.sqrt(2*9.81)
h10 = h20=qin_initial_ss**2/(a**2)/(2*9.81)
args=dict(Kc=0.01)
hsp = 1.1*h20

def dhdt(t, hvec, args):
    h1,h2=hvec
    qin=qin_initial_ss
    qcontrol= args['Kc']*(hsp - h20)
    q12 = a*sq2g*(h1**0.5)
    q2 = a*sq2g*(h2**0.5)
    return jnp.array( [(qin + qcontrol - q12)/A, (q12 - q2)/A])


dhdt_term = ODETerm(dhdt)
solver = Dopri5()


In [153]:
def h(t, y, args):
    return diffeqsolve(dhdt_term, solver, t0=0, t1=t, dt0=0.1, y0=jnp.array([h10, h20]), args=args).ys[0]

In [164]:
def loss_integrand(t,y,args):
    return (hsp-h(t,None,args)[1])

loss_integrand_term = ODETerm(loss_integrand)

def integrated_loss(args):
    integral=diffeqsolve(loss_integrand_term, solver, t0=0,t1=2., dt0=0.1, y0=jnp.array(0.), args=args)
    return integral.ys[0]

In [165]:
integrated_loss(args)

Array(1.0135515, dtype=float32, weak_type=True)

In [166]:
valgrad=jax.jit(jax.value_and_grad(integrated_loss))

In [167]:
args=dict(Kc=0.9)
valgrad(args)

(Array(1.0135515, dtype=float32, weak_type=True),
 {'Kc': Array(-0.00632896, dtype=float32, weak_type=True)})